In [14]:
from pathlib import Path

import numpy as np
import vedo

from histalign.backend.ccf.paths import get_atlas_path
from histalign.backend.io import load_volume
from histalign.backend.maths import compute_normal
from histalign.backend.models import (
    AlignmentSettings,
    HistologySettings,
    Orientation,
    Resolution,
    VolumeSettings,
)
from histalign.backend.workspace import VolumeSlicer, Volume
from notebook_helpers import show, update_axes, update_cameras, coronal_camera

In [15]:
def find_nearest_grid_point(point: np.ndarray) -> np.ndarray:
    return np.round(point)


def compute_stack_points(starting_point, normal, z_size, z_spacing, snap: bool = False):
    middle_index = (z_size - 1) // 2

    points = []
    for i in range(z_size):
        distance = (i - middle_index) * z_spacing
        point = starting_point + distance * normal
        if snap:
            point = find_nearest_grid_point(free_point)
        points.append(point)

    return points


def generate_aligned_planes(
    alignment_volume: Volume | vedo.Volume,
    alignment_paths: list[Path],
) -> list[vedo.Plane]:
    _module_logger.debug(
        f"Starting generation of {len(alignment_paths)} aligned planes."
    )

    planes = []
    slicer = VolumeSlicer(volume=alignment_volume)

    for index, alignment_path in enumerate(alignment_paths):
        if index > 0 and index % 5 == 0:
            _module_logger.debug(f"Generated {index} aligned planes...")

        alignment_settings = load_alignment_settings(alignment_path)
        histology_slice = load_image(alignment_settings.histology_path)

        registrator = Registrator(True, True)
        registered_slice = registrator.get_forwarded_image(
            histology_slice, alignment_settings
        )

        plane_mesh = slicer.slice(alignment_settings.volume_settings, return_mesh=True)

        # Undo padding. See `VolumeSlicer.slice` for more details.
        registered_slice = registered_slice[
            plane_mesh.metadata["i_padding"][0] : registered_slice.shape[0]
            - plane_mesh.metadata["i_padding"][1],
            plane_mesh.metadata["j_padding"][0] : registered_slice.shape[1]
            - plane_mesh.metadata["j_padding"][1],
        ]

        plane_mesh.pointdata["ImageScalars"] = registered_slice.flatten()

        planes.append(plane_mesh)

    _module_logger.debug(
        f"Finished generating all {len(alignment_paths)} aligned planes."
    )
    return planes


def insert_aligned_planes_into_array(
    array: np.ndarray,
    planes: list[vedo.Plane],
    inplace: bool = True,
) -> np.ndarray:
    _module_logger.debug(f"Starting insertion of {len(planes)} into alignment array.")

    if not inplace:
        array = array.copy()

    for index, plane in enumerate(planes):
        if index > 0 and index % 5 == 0:
            _module_logger.debug(f"Inserted {index} into alignment array...")

        temporary_volume = vedo.Volume(np.zeros_like(array))
        temporary_volume.interpolate_data_from(plane, radius=1)

        temporary_array = temporary_volume.tonumpy()
        temporary_array = np.round(temporary_array).astype(np.uint16)

        array[:] = np.maximum(array, temporary_array)

    return array


def build_aligned_volume(
    alignment_directory: str | Path,
    use_cache: bool = True,
    return_raw_array: bool = False,
) -> np.ndarray | vedo.Volume:
    if isinstance(alignment_directory, str):
        alignment_directory = Path(alignment_directory)

    alignment_paths = gather_alignment_paths(alignment_directory)
    if not alignment_paths:
        raise ValueError("Cannot build aligned volume from empty alignment directory.")

    alignment_hash = generate_hash_from_targets(alignment_paths)

    cache_path = ALIGNMENT_VOLUMES_CACHE_DIRECTORY / f"{alignment_hash}.npz"
    if cache_path.exists() and use_cache:
        _module_logger.debug("Found cached aligned volume. Loading from file.")

        array = np.load(cache_path)["array"]
        if return_raw_array:
            return array
        return vedo.Volume(array)

    reference_shape = load_alignment_settings(alignment_paths[0]).volume_settings.shape

    # Volume needs to be created before array as vedo makes a copy
    aligned_volume = vedo.Volume(np.zeros(shape=reference_shape, dtype=np.uint16))
    aligned_array = aligned_volume.tonumpy()

    planes = generate_aligned_planes(aligned_volume, alignment_paths)

    insert_aligned_planes_into_array(aligned_array, planes)
    aligned_volume.modified()  # Probably unnecessary but good practice

    if use_cache:
        _module_logger.debug("Caching volume to file as a NumPy array.")
        os.makedirs(ALIGNMENT_VOLUMES_CACHE_DIRECTORY, exist_ok=True)
        np.savez_compressed(cache_path, array=aligned_array)

    if return_raw_array:
        return aligned_array
    return aligned_volume

In [16]:
orientation = Orientation.CORONAL
resolution = Resolution.MICRONS_25

atlas_volume = load_volume(get_atlas_path(resolution))
update_cameras(atlas_volume.tonumpy().shape)
update_axes(atlas_volume.tonumpy().shape)

shape = atlas_volume.tonumpy().shape
z_stack_count = 21
z_stack_distance = 10

alignment_coordinates = np.array(shape) // 2
pitch = 10
yaw = 4

settings = AlignmentSettings(
    volume_path=get_atlas_path(resolution),
    volume_settings=VolumeSettings(
        orientation=orientation,
        resolution=resolution,
        pitch=pitch,
        yaw=yaw,
    ),
    histology_settings=HistologySettings(),
)

In [17]:
stack_shape = [*shape[1:], 10]
stack = np.random.randint(0, 2**16 - 1, size=np.prod(stack_shape)).reshape(stack_shape)

In [18]:
point_coordinates = compute_stack_points(
    alignment_coordinates, alignment_normal, z_stack_count, z_stack_distance
)

NameError: name 'alignment_normal' is not defined